In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
import os
if "notebooks" in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
from src.preprocessing.parser import parse_data

In [2]:
all_data = pd.read_csv(
    "data/combined_data_1.txt",
)

In [24]:
all_data.columns[0]

'1:'

In [4]:
movie_ratings_df = all_data\
    .reset_index()\
    .set_axis(['user_id', 'rating', 'date'], axis= 1)

In [5]:
movie_id = movie_ratings_df[movie_ratings_df['rating'].isna()]\
    .user_id

In [6]:
movie_id

547            2:
693            3:
2706           4:
2849           5:
3990           6:
            ...  
24046713    4495:
24047328    4496:
24056848    4497:
24057563    4498:
24057833    4499:
Name: user_id, Length: 4498, dtype: object

In [7]:
last_idx = movie_id.index[0]
complete_movie_ratings_list = []
for idx, value in tqdm(movie_id.iteritems()):
    if idx == movie_id.index[0] : 
        temp_df = movie_ratings_df.loc[0: idx]
        temp_df['movie_id'] = '1'    
    
    else:
        temp_df = movie_ratings_df.loc[last_idx: idx]
    
        temp_df['movie_id'] = temp_df.iloc[0]\
            ['user_id']\
            .replace(':', '')
    temp_df = temp_df.dropna()

    complete_movie_ratings_list.append(temp_df)
    last_idx = idx


4498it [00:50, 88.62it/s] 


In [8]:
features_df = pd.concat(complete_movie_ratings_list)

In [9]:
features_df

,user_id,rating,date,movie_id
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1
...,...,...,...,...
24057828,1184587,2.0,2004-11-02,4498
24057829,2349702,3.0,2002-07-22,4498
24057830,1174376,3.0,2005-03-03,4498
24057831,1305108,3.0,2002-02-03,4498


In [25]:
parsed = parse_data("data/combined_data_1.txt")

4498it [00:51, 88.12it/s] 


In [26]:
parsed

,user_id,rating,date,movie_id
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1
...,...,...,...,...
24057828,1184587,2.0,2004-11-02,4498
24057829,2349702,3.0,2002-07-22,4498
24057830,1174376,3.0,2005-03-03,4498
24057831,1305108,3.0,2002-02-03,4498


In [28]:
(parsed == features_df).sum()

user_id     24053336
rating      24053336
date        24053336
movie_id    24053336
dtype: int64

In [29]:
parsed.to_parquet("data/parsed/parsed_1.parquet")